In [1]:
#Install trnasformers library by huggingface
!pip install transformers

In [3]:
!pip install torch

In [4]:
import torch

In [5]:
# Loading a BERT model that has already been fine-tuned for the SQuAD benchmark.
# This model has been trained on version 1 of squad
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')



In [6]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [7]:
question = "How many parameters does BERT-large have?"
answer_text = "BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters! Altogether it is 1.34GB, so expect it to take a couple minutes to download to your Colab instance."

In [8]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_ids = tokenizer.encode(question, answer_text)

print('The input has a total of {:} tokens.'.format(len(input_ids)))

The input has a total of 70 tokens.


In [9]:
# BERT only needs the token IDs, but for the purpose of inspecting the 
# tokenizer's behavior, let's also get the token strings and display them.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')
    

[CLS]           101
how           2,129
many          2,116
parameters   11,709
does          2,515
bert         14,324
-             1,011
large         2,312
have          2,031
?             1,029

[SEP]           102

bert         14,324
-             1,011
large         2,312
is            2,003
really        2,428
big           2,502
.             1,012
.             1,012
.             1,012
it            2,009
has           2,038
24            2,484
-             1,011
layers        9,014
and           1,998
an            2,019
em            7,861
##bed         8,270
##ding        4,667
size          2,946
of            1,997
1             1,015
,             1,010
02            6,185
##4           2,549
,             1,010
for           2,005
a             1,037
total         2,561
of            1,997
340          16,029
##m           2,213
parameters   11,709
!               999
altogether   10,462
it            2,009
is            2,003
1             1,015
.             1,01

In [10]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)

In [18]:
torch.__version__

'1.8.1+cpu'

In [19]:
print(outputs)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-6.4849, -6.4358, -8.1077, -8.8489, -7.8751, -8.0522, -8.4684, -8.5295,
         -7.7074, -9.2464, -6.4849, -2.7303, -6.3473, -5.7299, -7.7780, -7.0391,
         -6.3331, -7.3153, -7.3048, -7.4121, -2.2534, -5.3971, -0.9424, -7.3584,
         -5.4575, -7.0769, -4.4887, -3.9272, -5.6967, -5.9506, -5.0059, -5.9812,
          0.0530, -5.5968, -4.7093, -4.5750, -6.1786, -2.2294, -0.1904, -0.2327,
         -2.7331,  6.4256, -2.6543, -4.5655, -4.9872, -4.9834, -5.9110, -7.8402,
         -1.8986, -7.2123, -4.1543, -6.2354, -8.0953, -7.2329, -6.4411, -6.8384,
         -8.1032, -7.0570, -7.7332, -6.8711, -7.1045, -8.2966, -6.1939, -8.0817,
         -7.5501, -5.9695, -8.1008, -6.8849, -8.2273, -6.4850]],
       grad_fn=<SqueezeBackward1>), end_logits=tensor([[-2.0629, -6.3878, -6.2450, -6.3605, -7.0722, -7.6281, -7.1160, -6.8674,
         -7.1313, -7.1495, -2.0628, -5.0858, -4.7276, -3.5955, -6.3050, -7.1109,
         -4.4975, -4.7221

In [11]:
# Run our example through the model.
outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                             token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                             return_dict=True) 

start_scores = outputs.start_logits
end_scores = outputs.end_logits


In [12]:
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

# Combine the tokens in the answer and print it out.
answer = ' '.join(tokens[answer_start:answer_end+1])

print('Answer: "' + answer + '"')

Answer: "340 ##m"


In [11]:
# Start with the first token.
answer = tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
for i in range(answer_start + 1, answer_end + 1):
    
    # If it's a subword token, then recombine it with the previous token.
    if tokens[i][0:2] == '##':
        answer += tokens[i][2:]
    
    # Otherwise, add a space then the token.
    else:
        answer += ' ' + tokens[i]

print('Answer: "' + answer + '"')

Answer: "340m"


In [13]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                    return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

In [14]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

# test_corpus = "We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial taskspecific architecture modifications. BERT is conceptually simple and empirically powerful. It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1 question answering Test F1 to 93.2 (1.5 point absolute improvement) and SQuAD v2.0 Test F1 to 83.1 (5.1 point absolute improvement)."

# test_corpus = "you can measure your emissions by calculating your carbon footprint, which is the total output of greenhouse gas emissions caused by an organisation, event,product or person. sahithi founded the most amazing place on called life. Sustainanility means utilizing the best of our resources to safeguard the planet."

# test_corpus = "emissions can be measured by calculating carbon footprint, which is the total output of greenhouse gas emissions caused by an organisation,event,product or person. Principles of sustainable corporate management are firmly anchored in the ZF strategy. The focus of sustainability efforts is placed on the four areas of products, production, employees, and supply chain. 'Our ambitious goal is to become climate-neutral by 2040 - ten years earlier than planned according to the Paris Agreement,' said ZF CEO Scheider. 'Among other things, we want to systematically cut our CO2 emissions, throughout our supply chain.' ZF has therefore introduced the sustaina ility criterion to raise awareness among business partners regarding its expectations in terms of sustainability and decarbonization. ZF has also joined the 'Alliance of CEO Climate Leaders' of the World Economic Forum because environmental protection can only be managed successfully in a joint effort."

# test_corpus = "In 2015, world leaders agreed to 17 Global Goals (officially known as the Sustainable Development Goals or SDGs). It's now five years on, and we have more work than ever to do. These goals have the power to create a better world by 2030, by ending poverty, fighting inequality and addressing the urgency of climate change. Guided by the goals, it is now up to all of us, governments, businesses, civil society and the general public to work together to build a better future for everyone."

# test_corpus = "The saudi arabian economy shrank by 3.9 percent year-on-year in the fourth quarter of 2020, slightly more than initial estimates of 3.8 percent drop."
# test_corpus = '''key performance indicators to measure the success of sustainability strategy include supply chain waste, energy waste, load density and greenhouse gas emissions.
# Sustainability measurement is the quantitative basis for the informed management of sustainability.
# The metrics used for the measurement of sustainability (involving the sustainability of environmental, social and economic domains, both individually and in various combinations) are still evolving: 
# they include indicators, benchmarks, audits, indexes and accounting, as well as assessment, appraisal and other reporting systems.emissions can be measured by calculating carbon footprint, which is the total
# output of greenhouse gas emissions caused by an organisation,event,product or
# person. Principles of sustainable corporate management are firmly anchored in
# the ZF strategy. The focus of sustainability efforts is placed on the four areas
# of products, production, employees, and supply chain. 'Our ambitious goal is to
# become climate-neutral by 2040 - ten years earlier than planned according to the
# Paris Agreement,' said ZF CEO Scheider. 'Among other things, we want to
# systematically cut our CO2 emissions, throughout our supply chain.' ZF has
# therefore introduced the sustaina ility criterion to raise awareness among
# business partners regarding its expectations in terms of sustainability and
# decarbonization. ZF has also joined the 'Alliance of CEO Climate Leaders' of the
# World Economic Forum because environmental protection can only be managed
# successfully in a joint effort. Goals for ZF include innovation and cost leadership, balanced market prediction, financial independance,globally attractive employer and diversified for profit.
# The target key performace indicators for ZF include ZF sales growth 1.5x Market growth,EBIT above average bench-mark,Regional Sales split: Europe 40%,Asia 30%,Americas 30%.
# '''



test_corpus = '''swedish cars have a strong reputation for vehicle and passenger safety. Not only that, the entire country is considered a model of road traffic safety worldwide. Statistically, the year 2015 saw only 2.4 traffic deaths per 100,000 inhabitants in Sweden, whereas the global average was 17.4 deaths, with 10.4 in the U.S. and still 4.3 in Germany. Sweden’s strong safety record can be traced back to its politicians, who paved the way for improved safety early on. In 1997, the Swedish Parliament laid the foundation for Vision Zero in its transport policy. The goal was to reduce to zero the number of deaths and severe injuries resulting from traffic accidents. Other countries – Germany among them – followed Sweden’s lead as did organizations like the European Union. By the year 2050, the EU plans to reach almost “zero loss of life” through traffic accidents.
 '''

print(wrapper.fill(test_corpus))

swedish cars have a strong reputation for vehicle and passenger safety. Not only
that, the entire country is considered a model of road traffic safety worldwide.
Statistically, the year 2015 saw only 2.4 traffic deaths per 100,000 inhabitants
in Sweden, whereas the global average was 17.4 deaths, with 10.4 in the U.S. and
still 4.3 in Germany. Sweden’s strong safety record can be traced back to its
politicians, who paved the way for improved safety early on. In 1997, the
Swedish Parliament laid the foundation for Vision Zero in its transport policy.
The goal was to reduce to zero the number of deaths and severe injuries
resulting from traffic accidents. Other countries – Germany among them –
followed Sweden’s lead as did organizations like the European Union. By the year
2050, the EU plans to reach almost “zero loss of life” through traffic
accidents.


In [15]:
question = "what is the goal by 2050"
answer_question(question, test_corpus)


Query has 190 tokens.

Answer: "by the year 2050 , the eu plans to reach almost “ zero loss of life ” through traffic accidents"


In [16]:
# "what are the KPIs to measure sucess of sustainability strategy"
# "what are the metrics used to measure sustainability"
# what is the main goal of ZF?"
# "what are the goals of zf?"
# "what are the target KPI's of ZF"
# "why has zf introduced the sustainablity criterion"
# "Why has ZF joined the alliance of CEO Climate Leaders"

















# why did ZF introduce sustainability criterion
# why did zf join 'Alliance of CEO Climate leaders
# what are the sustainability goals of ZF
# by when will you be carbon neutral
# what steps does ZF take towards sustainability
# why has zf committed to sustainability
# which sustainability organisation is zf a part of
# "50.9 pmi belongs to which month?"
# "Kenya's economy shrank 1.1% year-on-year in the third quarter of 2020, following a dwonwardly revised 5.5% plunge in the previous period."

In [17]:
question = "how are "

answer_question(question, test_corpus)

Query has 185 tokens.

Answer: "the entire country is considered a model of road traffic safety worldwide"
